# Neighborhoods in Toronto

### Import Data from Wiki

In [101]:
# imports
import pandas as pd 
import requests 
import numpy as np
from bs4 import BeautifulSoup 
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

# get the response in the form of html
wikiurl="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)


In [10]:
# parse data from the html into a beautifulsoup object
table_contents=[] #List created to pass the data
soup = BeautifulSoup(response.text, 'html.parser')
table_toronto=soup.find('table')

#Split the rows for the different columns 
#(ignoring cells with boroughs not assigned and concatenating with commas the neighborhood)
for row in table_toronto.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [14]:
# convert list to dataframe and replace values
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.shape


(103, 3)

### Adding the geolocalization via CSV shared

In [22]:
#import CSV
geodata = pd.read_csv('Geospatial_Coordinates.csv')

In [60]:
#merging dataframe with neighborhoods and geodata
table_geotoronto= df.merge(geodata, left_on='PostalCode', right_on='Postal Code')
table_geotoronto2 = table_geotoronto.drop(columns=['Postal Code'])
table_geotoronto2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


### Clustering

In [67]:
#Filter only the Borough that contains the word Toronto
table_toronto_borough = table_geotoronto2[table_geotoronto['Borough'].str.contains('Toronto')].reset_index(drop=True)
table_toronto_borough.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [68]:
#Visualize the boroughs from the previous table. Fisrt we need to import folium.
import folium

# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[43.682270, -79.366074], zoom_start=12)

# add markers to map
for lat, lng, label in zip(table_toronto_borough['Latitude'], table_toronto_borough['Longitude'], table_toronto_borough['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [74]:
#Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
#FourSquare Credential
CLIENT_ID = 'GBCPRLKKJ1BNCQ31M2JXN1YPKTZJ3N30SR2111KH44QEBUPP' # your Foursquare ID
CLIENT_SECRET = 'VNN5NS10RJDRMF3PS0EYLOQ10CJF2NGWXCDJ42WPPNZ0CKHT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 50 # A default Foursquare API limit value

In [82]:
#Evaluate Neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [83]:
#New Dataframe for the venues
toronto_venues = getNearbyVenues(names=table_toronto_borough['Neighborhood'],latitudes=table_toronto_borough['Latitude'],longitudes=table_toronto_borough['Longitude'])


Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [85]:
print(toronto_venues.shape)
toronto_venues.head()

(1145, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [86]:
# Analizing each Neighborhood
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.00,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.00000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.00,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.00,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,...,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000
3,Central Bay Street,0.020000,0.00,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.00000,0.000000
4,Christie,0.000000,0.00,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000
5,Church and Wellesley,0.040000,0.02,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.02,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000
6,"Commerce Court, Victoria Hotel",0.000000,0.00,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,...,0.00,0.00000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.00000,0.000000
7,Davisville,0.000000,0.00,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.00000,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000
8,Davisville North,0.000000,0.00,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.00,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.00,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000


In [88]:
# 5 most common venue from each neighborhood
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.06
1  Seafood Restaurant  0.04
2         Cheese Shop  0.04
3            Beer Bar  0.04
4        Cocktail Bar  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2          Bakery  0.09
3     Coffee Shop  0.09
4    Intersection  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3  Sculpture Garden  0.06
4           Airport  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1      Sandwich Place  0.06
2     Bubble Tea Shop  0.04
3                Café  0.04
4  Italian Restaurant  0.04


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3     Restaurant  0.06
4     Bab

In [91]:
# Create DataFrame with the 10 most common venues for each Neighborhood

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Restaurant,Bakery,Beer Bar,Cocktail Bar,Farmers Market,Cheese Shop,Beach,Bistro
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bakery,Nightclub,Burrito Place,Italian Restaurant,Intersection,Bar,Stadium
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Bar,Harbor / Marina,Boat or Ferry,Airport Gate,Airport Food Court,Airport
3,Central Bay Street,Coffee Shop,Sandwich Place,Bubble Tea Shop,Café,Italian Restaurant,Comic Shop,Salad Place,Burger Joint,Portuguese Restaurant,Poke Place
4,Christie,Grocery Store,Café,Park,Coffee Shop,Candy Store,Italian Restaurant,Baby Store,Athletics & Sports,Restaurant,Nightclub


In [97]:
# Clustering

#I would cluster into 4 groups
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [99]:
# create new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = table_toronto_borough

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Wine Shop,Event Space,Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Café,Ramen Restaurant,Cosmetics Shop,Clothing Store,Bookstore,Fast Food Restaurant,Theater,Mexican Restaurant,Pizza Place
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Gastropub,Coffee Shop,Cosmetics Shop,Park,Creperie,Seafood Restaurant,Bakery,Farmers Market,Camera Store
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Health Food Store,Pub,Wine Shop,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Seafood Restaurant,Restaurant,Bakery,Beer Bar,Cocktail Bar,Farmers Market,Cheese Shop,Beach,Bistro


In [102]:
#Visualize results

# create map
map_clusters = folium.Map(location=[43.682270, -79.366074], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [103]:
# Examine each cluster

#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,East York/East Toronto,0,Intersection,Park,Convenience Store,Wine Shop,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [104]:

#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Wine Shop,Event Space,Restaurant
1,Downtown Toronto,1,Coffee Shop,Café,Ramen Restaurant,Cosmetics Shop,Clothing Store,Bookstore,Fast Food Restaurant,Theater,Mexican Restaurant,Pizza Place
2,Downtown Toronto,1,Café,Gastropub,Coffee Shop,Cosmetics Shop,Park,Creperie,Seafood Restaurant,Bakery,Farmers Market,Camera Store
3,East Toronto,1,Trail,Health Food Store,Pub,Wine Shop,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
4,Downtown Toronto,1,Coffee Shop,Seafood Restaurant,Restaurant,Bakery,Beer Bar,Cocktail Bar,Farmers Market,Cheese Shop,Beach,Bistro
5,Downtown Toronto,1,Coffee Shop,Sandwich Place,Bubble Tea Shop,Café,Italian Restaurant,Comic Shop,Salad Place,Burger Joint,Portuguese Restaurant,Poke Place
6,Downtown Toronto,1,Grocery Store,Café,Park,Coffee Shop,Candy Store,Italian Restaurant,Baby Store,Athletics & Sports,Restaurant,Nightclub
7,Downtown Toronto,1,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Concert Hall,Steakhouse,American Restaurant,Thai Restaurant,Pizza Place,Restaurant,Burrito Place
8,West Toronto,1,Bakery,Pharmacy,Gym / Fitness Center,Supermarket,Bank,Bar,Middle Eastern Restaurant,Art Gallery,Café,Pet Store
10,Downtown Toronto,1,Brewery,Scenic Lookout,Park,Café,Pizza Place,Hotel,Aquarium,Plaza,Coffee Shop,History Museum


In [105]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,2,Park,Jewelry Store,Trail,Sushi Restaurant,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
29,Central Toronto,2,Park,Gym,Trail,Wine Shop,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
33,Downtown Toronto,2,Park,Playground,Trail,Wine Shop,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [106]:
#Cluster 4

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Park,Bus Line,Swim School,Business Service,Comfort Food Restaurant,Comic Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
